[![Binder](https://mybinder.org/badge_logo.svg)](https://gesis.mybinder.org/v2/gh/homalg-project/CapAndHomalgNotebooks/master?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fhomalg-project%252FFinSetsForCAP%26urlpath%3Dtree%252FFinSetsForCAP%252Fexamples%252F%252Fnotebooks%252Ftyped_lambda_calculus_and_category_theory.ipynb%26branch%3Dmaster)

Load required Julia and GAP packages:

In [1]:
using CapAndHomalg

CapAndHomalg v1.4.1
Imported OSCAR's components GAP and Singular_jll
Type: ?CapAndHomalg for more information


In [2]:
LoadPackage( "FinSetsForCAP" );

#### Convenience
Hide technical terms from category theory behind technical terms from lamda calculus:

In [3]:
NewContext = DirectProduct; # for a context and a type
FunctionType = ExponentialOnObjects; # for two types
EmptyContext = TerminalObject;
AsNamedConstant = CartesianLambdaIntroduction;
FreeVariableOfType = IdentityMorphism;

Application = function( source_type, range_type, function_type_judgement, term_type_judgement )
    
    @assert IsEqualForObjects( Range( function_type_judgement ), ExponentialOnObjects( source_type, range_type ) )
    @assert IsEqualForObjects( Range( term_type_judgement ), source_type )
    
    return PreCompose( DirectProductFunctorial( GapObj( [ function_type_judgement, term_type_judgement ] ) ), CartesianEvaluationMorphism( source_type, range_type ) );
    
end;

Abstraction = function( context, variable_type, term_type_judgement )
    
    @assert Source( term_type_judgement ) == DirectProduct( context, variable_type );
    
    return DirectProductToExponentialAdjunctionMap( context, variable_type, term_type_judgement );
    
end;

ConstantMapOfFinSets = function ( source, element, range )
    
    @assert element in AsList( range );
    
    return MapOfFinSets( source, List( source, GapObj( x -> GapObj( [ x, element ] ) ) ), range );
    
end;

#### Example: Objects and morphisms in the category FinSets

In [4]:
M = FinSet( [ 0, 1, 2 ] )

GAP: <An object in FinSets>

In [5]:
N = FinSet( [ 0, 1 ] )

GAP: <An object in FinSets>

In [6]:
f = MapOfFinSets( M, [ [0,1], [1,0], [2,0] ], N )

GAP: <A morphism in FinSets>

In [7]:
Display( IsWellDefined( f ) );

true


#### Example: Types and contexts

Choose some semantics/interpretation for the sets constructed above:

In [8]:
M; # the type of integers mod 3 (could also be the type of Apples, Bananas, Oranges)
N; # the type of integers mod 2

In [9]:
empty_context = EmptyContext( FinSets );

Construct a context with two free variables: the first of type "integers mod 3", the second of type "integers mod 2":

In [10]:
context_1_M_N = NewContext( NewContext( empty_context, M ), N );
Display( context_1_M_N );

[ [ [ "*", 0 ], 0 ], [ [ "*", 1 ], 0 ], [ [ "*", 2 ], 0 ], [ [ "*", 0 ], 1 ], [ [ "*", 1 ], 1 ], [ [ "*", 2 ], 1 ] ]


Construct a function type:

In [11]:
list_of_maps_M_to_N = AsList( FunctionType( M, N ) );
Perform( list_of_maps_M_to_N, Display );

[ [ 0, 1, 2 ], [ [ 0, 0 ], [ 1, 0 ], [ 2, 0 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 1 ], [ 1, 0 ], [ 2, 0 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 0 ], [ 1, 1 ], [ 2, 0 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 1 ], [ 1, 1 ], [ 2, 0 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 0 ], [ 1, 0 ], [ 2, 1 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 1 ], [ 1, 0 ], [ 2, 1 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 0 ], [ 1, 1 ], [ 2, 1 ] ], [ 0, 1 ] ]
[ [ 0, 1, 2 ], [ [ 0, 1 ], [ 1, 1 ], [ 2, 1 ] ], [ 0, 1 ] ]


#### Example: successor function

Define the successor function mod 3 as an external function:

In [12]:
external_successor_mod_3 = MapOfFinSets( M, [ [0,1], [1,2], [2,0] ], M );
@assert IsWellDefined( external_successor_mod_3 );

Import it as a named constant:

In [13]:
f = AsNamedConstant( external_successor_mod_3 );

In [14]:
function_type_M_M = FunctionType( M, M );

Named constants are morphisms from the empty context to a function type:

In [15]:
Display( Source( f ) == empty_context );
Display( Range( f ) == function_type_M_M );

true
true


Get a free variable of type M:

In [16]:
x = FreeVariableOfType( M );

Apply f to the free variable x, i.e. form the lambda term "fx":

In [17]:
fx = Application( M, M, f, x );

Type safety: The following would throw an error:

In [18]:
# Display( Application( M, M, f, FreeVariableOfType( N ) ) );

Bind the free variable x again by abstraction, i.e. form the lambda term "λx.fx":

In [19]:
lambda_x_fx = Abstraction( empty_context, M, fx );

In [20]:
Display( lambda_x_fx == f ); # eta-Konversion: "λx.fx == f"

true


Create λx.f(f(fx)):

In [21]:
f_fx = Application( M, M, f, fx );

Source( f_fx ) is 1 x (1 x M) which is equivalent to 1 x M via the left unitor:

In [22]:
f_fx = PreCompose( CartesianLeftUnitorInverse( NewContext( empty_context, M ) ), f_fx );

Now Source( f_fx ) is simplified to 1 x M.

In [23]:
f_f_fx = Application( M, M, f, f_fx );
f_f_fx = PreCompose( CartesianLeftUnitorInverse( NewContext( empty_context, M ) ), f_f_fx );

In [24]:
lambda_x_f_f_fx = Abstraction( empty_context, M, f_f_fx );

Recall that f was the successor function mod 3.
As expected, "λx.f(f(fx))" is equal to "λx.id_M x":

In [25]:
Display( lambda_x_f_f_fx == AsNamedConstant( IdentityMorphism( M ) ) );

true


#### Example: Church booleans
https://en.wikipedia.org/wiki/Church_encoding#Church_Booleans

In lambda calculus, booleans can be modeled as 

true = λa.λb.a

false = λa.λb.b

In *typed* lambda calculus, a and b must be of some type, so actually for every type we get "booleans on this type".


Construct true and false on N:

In [26]:
type_of_booleans_on_N = FunctionType( N, FunctionType( N, N ) );

In [27]:
true_on_N_external = MapOfFinSets(
    N,
    [
        [ 0, ConstantMapOfFinSets( N, 0, N ) ],
        [ 1, ConstantMapOfFinSets( N, 1, N ) ],
    ],
    FunctionType( N, N )
);
@assert IsWellDefined( true_on_N_external )

In [28]:
false_on_N_external = ConstantMapOfFinSets( N, IdentityMorphism( N ), FunctionType( N, N ) );
@assert IsWellDefined( false_on_N_external )

Construct the predicate "is_zero" on N:

In [29]:
predicate_is_zero_on_N_external = MapOfFinSets(
    N,
    [
        [ 0, true_on_N_external ],
        [ 1, false_on_N_external ],
    ],
    type_of_booleans_on_N
);

Import it as a named constant:

In [30]:
is_zero = AsNamedConstant( predicate_is_zero_on_N_external );

Now we want to model the following function on N = {0,1}:
```
function ( x )
    if x == 0 then
        return 1;
    else
        return 0;
    fi;
end;
```

In [31]:
x = FreeVariableOfType( N );

Construct the values 0 and 1 in N:

(I'm not sure if/how this generalizes to settings where the empty context is not a terminal object)

In [32]:
zero_in_N = MapOfFinSets( empty_context, [ [ AsList( empty_context )[1], 0 ] ], N );
one_in_N = MapOfFinSets( empty_context, [ [ AsList( empty_context )[1], 1 ] ], N );

By construction, is_zero_x can be applied to two values: if x is zero, the first value is returned, else the second value:

In [33]:
is_zero_x = Application( N, type_of_booleans_on_N, is_zero, x );

In [34]:
is_zero_x_1 = Application( N, FunctionType( N, N ), is_zero_x, one_in_N );

As above, simplify source by composing with the correct unitor:

In [35]:
is_zero_x_1 = PreCompose( CartesianRightUnitorInverse( NewContext( empty_context, N ) ), is_zero_x_1 );

In [36]:
is_zero_x_1_0 = Application( N, N, is_zero_x_1, zero_in_N );
is_zero_x_1_0 = PreCompose( CartesianRightUnitorInverse( NewContext( empty_context, N ) ), is_zero_x_1_0 );

In [37]:
lambda_x_is_zero_x_1_0 = Abstraction( empty_context, N, is_zero_x_1_0 );

Check that lambda_x_is_zero_x_1_0 indeed is the function swapping 0 and 1 as expected:

In [38]:
external_swap = MapOfFinSets( N, [ [0,1], [1,0] ], N );
Display( lambda_x_is_zero_x_1_0 == AsNamedConstant( external_swap ) );

true
